In [6]:
import os
import pandas as pd
import datetime
from collections import defaultdict

# Check Entire Data

#### Folder
 - "../data_backup/": Top 50 marketcap  
 - "../data/": BTC/BUSD


### Test Count

In [7]:
path = "../data/"
file_list = os.listdir(path)

count_shape=defaultdict(int)

file_shape_short = []
file_shape_normal = []
file_shape_over = []
m6,y1,y2=4344,17520//2,17520
cnt=m6

for file in file_list:
    if ".csv" not in file:
        continue
    # if "REPAIRED" not in file:
    #     continue

    df=pd.read_csv(path+file, names=['datetime', 'open', 'high', 'low', 'close', 'volume'])

    if df.shape[0] < cnt:
        count_shape["Short"]+=1
        file_shape_short.append(file)

    if df.shape[0] == cnt:
        count_shape["Normal"]+=1
        file_shape_normal.append(file)

    if df.shape[0] > cnt:
        count_shape["Over"]+=1
        file_shape_over.append(file)
    print(df.shape)


# Print Result
for k,v in count_shape.items():
    print("%7s - %3d"%(k,v))
print("%7s - %3d"%("Total",len(file_list)))

(4380, 6)
(4380, 6)
(43, 6)
   Over -   2
  Short -   1
  Total -   3


### Test Duplicate

In [8]:
file_list = file_shape_over + file_shape_normal

count_duplicate=defaultdict(int)

file_duplicate_pass = []
file_duplicate_fail = []

for file in file_list:

    df=pd.read_csv(path+file, names=['datetime', 'open', 'high', 'low', 'close', 'volume'])

    if df[df.duplicated(['datetime'])].shape[0]:
        count_duplicate["Pass"]+=1
        file_duplicate_pass.append(file)

    else:
        count_duplicate["Fail"]+=1
        file_duplicate_fail.append(file)

# Print Result
for k,v in count_duplicate.items():
    print("%7s - %3d"%(k,v))
print("%7s - %3d"%("Total",len(file_list)))
#  Fail -  22
#  Total -  22

   Fail -   2
  Total -   2


### Test Diff

In [9]:
file_list = file_duplicate_fail

count_diff=defaultdict(int)

file_diff_short = []
file_diff_normal = []
file_diff_over = []

file_diff_pass = []
file_diff_fail = []


for file in file_list:

    df=pd.read_csv(path+file, names=['datetime', 'open', 'high', 'low', 'close', 'volume'])
    # Resolve Duplicate
    df.drop_duplicates("datetime",inplace=True)
    
    df=df[df['datetime']<1656633600000]
    print(df.shape)
    if df.shape[0] < cnt:
        file_diff_short.append(file)
        continue

    elif df.shape[0] == cnt:
        file_diff_normal.append(file)
        
    else:
        file_diff_over.append(file)
        continue

    df.sort_values(by=['datetime'],inplace=True)
    df['diff']=df['datetime'].diff()

    if df[df['diff'] != 3600000.0].shape[0] < 2: # normal
        count_diff["Pass"]+=1
        file_diff_pass.append(file)
    else:
        count_diff["Fail"]+=1
        file_diff_fail.append(file)

# Print Result
for k,v in count_diff.items():
    print("%7s - %3d"%(k,v))
print("%7s - %3d"%("Total",len(file_list)))

(4118, 6)
(4344, 6)
   Pass -   1
  Total -   2


In [10]:
with open("file_diff_pass.txt", 'w') as f:
    for symbol in file_diff_pass:
        f.write("%s\n"%symbol)

# Check Single File

In [61]:
path = "../data/"

file="BINANCE_CELOBUSD_1h.csv"

df=pd.read_csv(path+file, names=['datetime', 'open', 'high', 'low', 'close', 'volume'])
df=df[df['datetime']<1656633600000]
df['datetime']=df['datetime'].apply(lambda x: datetime.datetime.utcfromtimestamp(int(x/1000)))
df['diff']=df['datetime'].diff()
df

,datetime,open,high,low,close,volume,diff
0,2022-01-01 00:00:00,4.870,5.022,4.870,4.995,19904.4,NaT
1,2022-01-01 01:00:00,4.992,5.112,4.974,5.074,14525.3,0 days 01:00:00
2,2022-01-01 02:00:00,5.082,5.153,5.070,5.133,12909.5,0 days 01:00:00
3,2022-01-01 03:00:00,5.133,5.148,5.100,5.100,14523.2,0 days 01:00:00
4,2022-01-01 04:00:00,5.109,5.181,5.029,5.066,21097.8,0 days 01:00:00
...,...,...,...,...,...,...,...
4339,2022-06-30 19:00:00,0.831,0.837,0.828,0.834,14504.6,0 days 01:00:00
4340,2022-06-30 20:00:00,0.833,0.837,0.822,0.830,25038.0,0 days 01:00:00
4341,2022-06-30 21:00:00,0.831,0.838,0.831,0.833,14648.2,0 days 01:00:00
4342,2022-06-30 22:00:00,0.833,0.845,0.828,0.839,11867.4,0 days 01:00:00


# DEPRECATED

In [ ]:

key_list=["Normal","Abnormal","Fail", "Duplicate"]
for key in key_list:
    print("%8s:\t%2d"%(key, len(data_dict[key])))

In [29]:
# for check data count
start_date='20220101'
end_date = '20220702'
data_list=pd.date_range(start=start_date,end=end_date,freq='h')
